In [2]:
from bertopic import BERTopic
import pandas as pd
import json
import os

In [4]:
# We have 2 tasks in this code:
# 1) Gather topic_ids in a dict, with each topic_id's value being another dict.
#     The inner dict contains that topic_id's score in each book.
# 2) Make a dict of all words used in these topics, with the values being their topic_id.

In [14]:
load_path = "../data/interim/topics/"
save_path = "../data/final/"

In [78]:
# get BERTopic model info
model = BERTopic.load("MaartenGr/BERTopic_Wikipedia")
df_fulltopic = model.get_topic_info()
df = df_fulltopic[["Topic", "Representation"]]
df = df.drop([0])  # remove topic -1

In [96]:
# for i in df.index:
#     if 'cast' in df["Representation"][i]:
#         print(df["Topic"][i])
#         print(df["Representation"][i])

# # df["Representation"][2][:5]

In [32]:
full_vocab_dict = {}
full_vocab_dict_reversed = {}

for i in df.index:
    topic_id, words = df["Topic"][i], tuple(df["Representation"][i])
    full_vocab_dict[topic_id] = words

for k, v in full_vocab_dict.items():
    full_vocab_dict_reversed[v] = k

In [35]:
# Task 1

In [86]:
ids = []
for f in os.listdir(load_path):
    if f.endswith(".csv"):
        ids.append(f[:3])
ids.sort()

topic_dict = {}

for id in ids:
    f = id + ".csv"
    df_book = pd.read_csv(load_path + f)

    # normalize probabilities 
    tot = sum(df_book["prob"])
    df_book["prob"] /= tot

    try:
        for i in df_book.index:
            p, words = df_book["prob"][i], tuple(eval(df_book["words"][i])[0])
            topic_id = full_vocab_dict_reversed[words]

            topic_dict.setdefault(str(topic_id), {})[id] = p

    except:
        # super weird, one topic is present in book topics
        # but I can't find it in BERTopic_wikipedia's topic info
        # even though that's the model that was used to generate
        # the book topics. no clue why.
        pass
        # print(words)
        # print(id)

In [87]:
# write topics dict
out_file = save_path + "topics.txt"
with open(out_file, "w") as file:
    json.dump(topic_dict, file)

In [83]:
# Task 2

In [92]:
vocab = {}

for topic_id in topic_dict:
    words = full_vocab_dict[int(topic_id)]
    for word in words:
        vocab.setdefault(word, []).append(topic_id)

In [95]:
# write vocab mapping file
out_file = save_path + "vocab.txt"
with open(out_file, "w") as file:
    json.dump(vocab, file)